<a href="https://colab.research.google.com/github/roque-alfaro/taller-eiv-2026/blob/main/4_Ejercicio_visualizaciones_y_reportes_(Python).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejercicio: construir gráficos interactivos

En este ejercicio vamos a visualizar los datos de demanda y oferta de consultas de especialidad médica y reproducir los [análisis de brechas en el SSMSO 2021](https://www.revistamedicadechile.cl/index.php/rmedica/article/view/10435). Para ello necesitamos:

1. Cargar el cubo de datos
2. Hacer un análisis exploratorio de datos
  - Crear una tabla de contingencia (oferta vs demanda por establecimiento y año)
3. Crear una función para graficar las series de tiempo
  - Filtrar por especialidad
  - Estratificar por establecimiento
4. Graficar las brechas de consultas

# Configurar área de trabajo y cargar datos
Usaremos las librerías de [Plotly](https://plotly.com/graphing-libraries/) para construir gráficos interactivos 📊✨.

In [23]:
# importar librerías
import pandas as pd
import os
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# definir carpetas de trabajo
datos_multidimensionales = 'https://github.com/rlagosb/taller_eiv/raw/refs/heads/main/datos_multidimensionales/'

In [24]:
# cargar datos
cubo = pd.read_excel(datos_multidimensionales + 'Cubo_consultas_nuevas.xlsx')

## 🏁 Discusión

1. ¿Qué métricas y dimensiones tiene esta tabla?
2. ¿Cuántas observaciones por año contiene?

In [25]:
cubo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3776 entries, 0 to 3775
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Periodo                 3776 non-null   object
 1   Año                     3776 non-null   int64 
 2   Mes                     3776 non-null   int64 
 3   Trimestre               3776 non-null   object
 4   Centro_cod              3776 non-null   int64 
 5   Especialidad_cod        3776 non-null   object
 6   Consultas_producidas    3776 non-null   int64 
 7   Consultas_inasistencia  3776 non-null   int64 
 8   Oferta_consultas        3776 non-null   int64 
 9   Consultas_solicitadas   3776 non-null   int64 
 10  Lista_espera_inicial    3776 non-null   int64 
 11  Especialidad            3776 non-null   object
 12  Riesgo mortalidad       3776 non-null   object
 13  Centro                  3776 non-null   object
 14  Centro_siglas           3776 non-null   object
 15  Serv

## 🍎 Desafío

Cree una tabla de contingencia para reportar la oferta y demanda por año y especialidad

In [26]:
tabla_contingencia = (
    cubo
    .groupby(["Año", "Especialidad"], as_index=False)
    .agg(
        Demanda=("Consultas_solicitadas", "sum"),
        Oferta_producida=("Consultas_producidas", "sum"),
        Oferta_programada=("Oferta_consultas", "sum")  # opcional
    )
)

# (opcional) brechas
tabla_contingencia["Brecha_producida"] = tabla_contingencia["Demanda"] - tabla_contingencia["Oferta_producida"]
tabla_contingencia["Brecha_programada"] = tabla_contingencia["Demanda"] - tabla_contingencia["Oferta_programada"]

tabla_contingencia.head()


,Año,Especialidad,Demanda,Oferta_producida,Oferta_programada,Brecha_producida,Brecha_programada
0,2021,ANESTESIOLOGIA,1049,3249,3498,-2200,-2449
1,2021,CARDIOLOGIA,6468,4964,5853,1504,615
2,2021,CARDIOLOGIA PEDIATRICA,1029,1160,1448,-131,-419
3,2021,CIRUGIA CARDIOVASCULAR,323,468,490,-145,-167
4,2021,"CIRUGIA DE CABEZA, CUELLO Y MAXILOFACIAL",223,2168,2385,-1945,-2162


In [27]:
tabla_pivot = tabla_contingencia.pivot_table(
    index="Especialidad",
    columns="Año",
    values=["Demanda", "Oferta_producida", "Brecha_producida"],
    aggfunc="sum"
)
tabla_pivot


Brecha_producida        \
Año                                                            2021  2022   
Especialidad                                                                
ANESTESIOLOGIA                                                -2200  -866   
CARDIOLOGIA                                                    1504  -762   
CARDIOLOGIA PEDIATRICA                                         -131  -160   
CIRUGIA CARDIOVASCULAR                                         -145   -70   
CIRUGIA DE CABEZA, CUELLO Y MAXILOFACIAL                      -1945  -686   
CIRUGIA DE TORAX                                               -125  -149   
CIRUGIA DIGESTIVA                                              -450  -540   
CIRUGIA GENERAL                                                -374 -1620   
CIRUGIA PEDIATRICA                                            -1165  -459   
CIRUGIA PLASTICA Y REPARADORA                                   -20   -91   
CIRUGIA PLASTICA Y REPARADORA PEDIATRICA                       -253  -184   
CIRUGIA VASCULAR PERIFERICA                                    -354  -500   
COLOPROCTOLOGIA                                                -599  -669   
DERMATOLOGIA                                                   -220  -957   
DIABETOLOGIA                                                   -914  -281   
ENDOCRINOLOGIA ADULTO                                           126  -339   
ENDOCRINOLOGIA PEDIATRICA                                      -104  -193   
ENFERMEDADES RESPIRATORIAS DEL ADULTO (BRONCOPU...              457  -397   
ENFERMEDADES RESPIRATORIAS PEDIATRICAS (BRONCOP...             -175  -349   
GASTROENTEROLOGIA ADULTO                                       1555   -57   
GASTROENTEROLOGIA PEDIATRICA                                     31  -224   
GENETICA CLINICA                                                 44  -144   
GERIATRIA                                                        58   -20   
GINECOLOGIA                                                     -57 -1919   
GINECOLOGIA PEDIATRICA Y DE LA ADOLESCENCIA                     104  -116   
HEMATO-ONCOLOGIA PEDIATRICA                                      41   -69   
HEMATOLOGIA                                                      58  -330   
IMAGENOLOGÍA                                                      0     0   
INFECTOLOGIA                                                    471   -90   
INFECTOLOGIA PEDIATRICA                                        -721  -238   
INMUNOLOGIA                                                    -136   -99   
MEDICINA FAMILIAR                                              -248  -222   
MEDICINA FAMILIAR DEL NIÑO                                     -416  -160   
MEDICINA FISICA Y REHABILITACION (FISIATRIA ADU...             -266  -675   
MEDICINA FISICA Y REHABILITACION PEDIATRICA (FI...               44    12   
MEDICINA INTERNA                                              -1591 -1444   
MEDICINA NUCLEAR                                                  0     0   
NEFROLOGIA ADULTO                                              -370  -452   
NEFROLOGIA PEDIATRICO                                          -208  -170   
NEONATOLOGIA                                                    -83   -59   
NEUROCIRUGIA                                                    240  -360   
NEUROLOGIA ADULTO                                             -1705 -1384   
NEUROLOGIA PEDIATRICA                                           -78  -647   
NUTRIOLOGO                                                      784    -3   
NUTRIOLOGO PEDIATRICO                                          -228  -106   
OBSTETRICIA                                                   -1721  -958   
OFTALMOLOGIA                                                  -5582 -3387   
ONCOLOGIA MEDICA                                               -866  -559   
OTORRINOLARINGOLOGIA                                          -1707 -1216   
PEDIATRIA                                      

In [28]:
tabla_contingencia = (
    cubo
    .groupby(["Año", "Especialidad"], as_index=False)
    .agg(
        Demanda=("Consultas_solicitadas", "sum"),
        Oferta=("Consultas_producidas", "sum")
    )
)

tabla_contingencia.head()


,Año,Especialidad,Demanda,Oferta
0,2021,ANESTESIOLOGIA,1049,3249
1,2021,CARDIOLOGIA,6468,4964
2,2021,CARDIOLOGIA PEDIATRICA,1029,1160
3,2021,CIRUGIA CARDIOVASCULAR,323,468
4,2021,"CIRUGIA DE CABEZA, CUELLO Y MAXILOFACIAL",223,2168


# Graficar series de tiempo
Construyamos un gráfico de la serie de tiempo con interactividad básica

In [29]:
# Veamos que necesitamos procesar los datos para visualizarlos adecuadamente

# prompt: genera un gráfico de Oferta_consultas por periodo desglosado por Centro_siglas

# Generar el gráfico
fig = px.line(cubo,
              x="Periodo",
              y="Oferta_consultas",
              color="Centro_siglas",
              title="Oferta de Consultas por Periodo y Centro")

fig.show()

## 🏁 Discusión
¿Porqué se ve así la serie de tiempo *Oferta_consultas*?

In [30]:
serie = cubo.groupby(['Periodo','Centro_siglas']).agg({'Oferta_consultas':'sum'}).reset_index()

# Generar el gráfico
fig = px.line(serie,
              x="Periodo",
              y="Oferta_consultas",
              color="Centro_siglas",
              title="Oferta de Consultas por Periodo y Centro")

fig.show()

## Gráfico interactivo

Para explorar las series, en vez de generar un script y un extracto para visualizar cada serie, podemos crear una función que reciba como parámetro la serie que queremos visualizar.

In [31]:
# Generamos una función que toma como parámetro el nombre de la serie:
# 'Consultas_producidas', 'Consultas_inasistencia', 'Oferta_consultas', 'Consultas_solicitadas', 'Lista_espera_inicial'

def graficar_serie(serie):

  df = cubo.groupby(['Periodo','Centro_siglas']).agg({serie:'sum'}).reset_index()

  # Generar el gráfico
  fig = px.line(df,
                x="Periodo",
                y=serie,
                color="Centro_siglas",
                title=serie+'  por Periodo y Centro')
  fig.show()


In [32]:
# Ejecutamos la función

graficar_serie('Consultas_producidas')

In [33]:
# También podemos agregar un parámetro que permita filtrar por especialidad

def graficar_serie(serie, especialidad):

  # generamos una variable para filtrar por especialidad
  filtro_especialidad = (cubo['Especialidad']==especialidad)
  df = cubo[filtro_especialidad].copy()

  df = cubo.groupby(['Periodo','Centro_siglas']).agg({serie:'sum'}).reset_index()
  if serie == 'Lista_espera_inicial': df[serie] = df[serie]/3     # métrica no es aditiva al agregar por trimestre (3 meses)


  # Generar el gráfico
  titulo = serie+'  por Periodo y Centro ('+especialidad+')'
  fig = px.line(df,
                x="Periodo",
                y=serie,
                color="Centro_siglas",
                title=titulo)
  fig.show()


In [34]:
# Ejecutamos la función para un caso

graficar_serie('Oferta_consultas', 'DERMATOLOGIA')

## 🍎 Desafíos

1. Explore otras series de tiempo cambiando los parámetros serie y/o especialidad. Ejecutando `cubo.Especialidad.unique()` puede obtener el listado de las especialidades
2. Agregue un parámetro a la función *graficar_serie* que permita filtrar los datos para un año específico

In [35]:
import plotly.express as px

def graficar_serie(serie, especialidad=None, anio=None):

    df = cubo.copy()

    # Filtrar por especialidad (si se entrega)
    if especialidad is not None:
        df = df[df["Especialidad"] == especialidad]

    # Filtrar por año (si se entrega)
    if anio is not None:
        df = df[df["Año"] == anio]

    # Agrupar para serie de tiempo por centro
    df = df.groupby(["Periodo", "Centro_siglas"], as_index=False).agg({serie: "sum"})

    # Ajuste especial para lista de espera (según pauta)
    if serie == "Lista_espera_inicial":
        df[serie] = df[serie] / 3

    # Título dinámico
    titulo = f"{serie} por Periodo y Centro"
    if especialidad is not None:
        titulo += f" ({especialidad})"
    if anio is not None:
        titulo += f" - Año {anio}"

    fig = px.line(
        df,
        x="Periodo",
        y=serie,
        color="Centro_siglas",
        title=titulo
    )
    fig.show()


In [36]:
graficar_serie("Oferta_consultas")
graficar_serie("Oferta_consultas", especialidad="DERMATOLOGIA")
graficar_serie("Oferta_consultas", especialidad="DERMATOLOGIA", anio=2021)
graficar_serie("Consultas_solicitadas", anio=2021)


In [37]:
cubo["Especialidad"].unique()


array(['ANESTESIOLOGIA', 'CARDIOLOGIA', 'CIRUGIA GENERAL',
       'CIRUGIA DE CABEZA, CUELLO Y MAXILOFACIAL',
       'CIRUGIA CARDIOVASCULAR', 'CIRUGIA DE TORAX',
       'CIRUGIA PLASTICA Y REPARADORA', 'CIRUGIA PEDIATRICA',
       'CIRUGIA VASCULAR PERIFERICA', 'COLOPROCTOLOGIA', 'DERMATOLOGIA',
       'DIABETOLOGIA', 'ENDOCRINOLOGIA ADULTO',
       'ENDOCRINOLOGIA PEDIATRICA',
       'ENFERMEDADES RESPIRATORIAS DEL ADULTO (BRONCOPULMONAR)',
       'ENFERMEDADES RESPIRATORIAS PEDIATRICAS (BRONCOPULMONAR PEDIATRICO)',
       'GASTROENTEROLOGIA ADULTO', 'GASTROENTEROLOGIA PEDIATRICA',
       'GENETICA CLINICA', 'GERIATRIA',
       'GINECOLOGIA PEDIATRICA Y DE LA ADOLESCENCIA', 'HEMATOLOGIA',
       'IMAGENOLOGÍA', 'INFECTOLOGIA', 'INMUNOLOGIA', 'MEDICINA FAMILIAR',
       'MEDICINA FISICA Y REHABILITACION (FISIATRIA ADULTO)',
       'MEDICINA INTERNA', 'MEDICINA NUCLEAR', 'NEFROLOGIA ADULTO',
       'NEFROLOGIA PEDIATRICO', 'NEONATOLOGIA', 'NEUROCIRUGIA',
       'NEUROLOGIA ADULTO', 'NE

## Desglosar por establecimiento

In [38]:
# prompt: modify 'graficar_series' so that each chart is next to each other sharing y axis

def graficar_serie_especialidad(serie, especialidad, anio):

  # generamos una variable para filtrar por especialidad y otra para el año
  filtro_especialidad = (cubo['Especialidad']==especialidad)
  filtro_año = (cubo['Año']==anio)
  df = cubo[filtro_especialidad & filtro_año].copy()

  # Obtenemos el listado de centros en 'Centro_siglas'
  centros = df['Centro_siglas'].unique()

  # Generamos una grilla de gráficos
  fig = make_subplots(rows=1, cols=len(centros), shared_yaxes=True,
                      subplot_titles=[f"{centro}" for centro in centros])

  # Generar serie por hospital y agregarlo a la grilla
  for i, centro in enumerate(centros):
    df_centro = df[df['Centro_siglas'] == centro].copy()

    # Generamos la serie
    df_centro = df_centro.groupby(['Periodo']).agg({serie:'sum'}).reset_index()
    if serie == 'Lista_espera_inicial': df_centro[serie] = df_centro[serie]/3

    fig.add_trace(go.Scatter(x=df_centro['Periodo'], y=df_centro[serie],
                             mode='lines+markers', name=centro),
                  row=1, col=i+1)

  fig.update_layout(title_text=f"{serie} por Periodo ({especialidad} {anio})",
                    yaxis_range=[0, None])
  fig.show()

In [39]:
graficar_serie_especialidad('Consultas_solicitadas', 'DERMATOLOGIA', 2021)

## 🏁 Discusión

1. ¿Qué ventajas y desventajas tiene generar las visualizaciones con un lenguaje de programación vs. realizarlos con Excel?
2. ¿Qué ventajas y desventajas tiene analizar los datos utilizando funciones como *graficar_series*?

# Graficar brechas

In [40]:
def graficar_brechas(especialidad, anio):

  # generamos variables para filtrar por especialidad y año
  filtro_especialidad = (cubo['Especialidad']==especialidad)
  filtro_anio = (cubo['Año']==anio)
  df = cubo[filtro_especialidad & filtro_anio].copy()

  df = df.groupby(['Periodo']).agg({'Oferta_consultas':'sum',
                                    'Consultas_solicitadas':'sum',
                                    'Lista_espera_inicial':'sum'}).reset_index()
  df['Lista_espera_inicial'] = df['Lista_espera_inicial']/3

  # Agregamos la Demanda y Oferta acumulada
  df['Demanda_acumulada'] = df['Consultas_solicitadas'].cumsum()
  df['Oferta_acumulada'] = df['Oferta_consultas'].cumsum()

  # Generar el gráfico
  fig = go.Figure()
  fig.add_trace(go.Scatter(x=df['Periodo'], y=df['Lista_espera_inicial'],
                         fill='tozeroy', name='Lista_espera_inicial',stackgroup='one'))
  fig.add_trace(go.Scatter(x=df['Periodo'], y=df['Demanda_acumulada'],
                         fill='tozeroy', name='Demanda_acumulada',stackgroup='one'))
  fig.add_trace(go.Scatter(x=df['Periodo'], y=df['Oferta_acumulada'],
                         mode='lines+markers', name='Oferta_acumulada'))
  fig.update_layout(title_text=f"Brechas por Periodo ({especialidad} {anio})")
  fig.show()

In [41]:
graficar_brechas('PSIQUIATRIA ADULTO',2021)

Finalmente generamos una función para desglosar las brechas por establecimiento

In [42]:
def graficar_brechas_centros(especialidad, anio):

  # generamos una variable para filtrar por especialidad y otra para el año
  filtro_especialidad = (cubo['Especialidad']==especialidad)
  filtro_anio = (cubo['Año']==anio)
  df = cubo[filtro_especialidad & filtro_anio].copy()

  df = df.groupby(['Trimestre','Centro_siglas']).agg({'Oferta_consultas':'sum',
                                    'Consultas_solicitadas':'sum',
                                    'Lista_espera_inicial':'first'}).reset_index()

  # Obtenemos el listado de centros en 'Centro_siglas'
  centros = df['Centro_siglas'].unique()

  # Generamos una grilla de gráficos
  fig = make_subplots(rows=1, cols=len(centros), shared_yaxes=True,
                      subplot_titles=[f"{centro}" for centro in centros])

  # Generar gráfico por hospital y agregarlo a la grilla
  for i, centro in enumerate(centros):

      # Calculamos la Demanda y Oferta acumulada
      df_centro = df[df['Centro_siglas'] == centro].copy()
      df_centro.loc[:,'Demanda_acumulada'] = df_centro['Consultas_solicitadas'].cumsum()
      df_centro.loc[:,'Oferta_acumulada'] = df_centro['Oferta_consultas'].cumsum()

      # Generamos los gráficos
      fig.add_trace(go.Scatter(x=df_centro['Trimestre'], y=df_centro['Oferta_acumulada'],
                             mode='lines+markers', name='Oferta_acumulada',
                               line_color='red',showlegend=(i==0)),
                  row=1, col=i+1)
      fig.add_trace(go.Scatter(x=df_centro['Trimestre'], y=df_centro['Lista_espera_inicial'],
                             fill='tozeroy', name='Lista_espera_inicial',
                               stackgroup='one',line_color='darkviolet',
                               showlegend=(i==0)),
                  row=1, col=i+1)
      fig.add_trace(go.Scatter(x=df_centro['Trimestre'], y=df_centro['Demanda_acumulada'],
                             fill='tozeroy', name='Demanda_acumulada',
                               stackgroup='one',line_color='violet',
                               showlegend=(i==0)),
                  row=1, col=i+1)

  # Agregamos título y publicamos
  fig.update_layout(title_text=f"Brechas por Periodo ({especialidad} {anio})")
  fig.show()


In [43]:
# Construimos el gráfico para una especialidad

graficar_brechas_centros('PSIQUIATRIA ADULTO',2021)

## 🍎 Desafíos
1. Ejecute *generar_brechas_centros* para otra especialidad
  - Ejecutando `cubo.Especialidad.unique()` puede obtener el listado de las especialidades
1. Modifique alguna de las visualizaciones incorporando otras variables disponibles en el cubo.
2. Genere otra visualización de las brechas que le parezca interesante para contrastar la demanda y oferta.

In [45]:
sorted(cubo["Especialidad"].dropna().unique())[:50]


['ANESTESIOLOGIA',
 'CARDIOLOGIA',
 'CARDIOLOGIA PEDIATRICA',
 'CIRUGIA CARDIOVASCULAR',
 'CIRUGIA DE CABEZA, CUELLO Y MAXILOFACIAL',
 'CIRUGIA DE TORAX',
 'CIRUGIA DIGESTIVA',
 'CIRUGIA GENERAL',
 'CIRUGIA PEDIATRICA',
 'CIRUGIA PLASTICA Y REPARADORA',
 'CIRUGIA PLASTICA Y REPARADORA PEDIATRICA',
 'CIRUGIA VASCULAR PERIFERICA',
 'COLOPROCTOLOGIA',
 'DERMATOLOGIA',
 'DIABETOLOGIA',
 'ENDOCRINOLOGIA ADULTO',
 'ENDOCRINOLOGIA PEDIATRICA',
 'ENFERMEDADES RESPIRATORIAS DEL ADULTO (BRONCOPULMONAR)',
 'ENFERMEDADES RESPIRATORIAS PEDIATRICAS (BRONCOPULMONAR PEDIATRICO)',
 'GASTROENTEROLOGIA ADULTO',
 'GASTROENTEROLOGIA PEDIATRICA',
 'GENETICA CLINICA',
 'GERIATRIA',
 'GINECOLOGIA',
 'GINECOLOGIA PEDIATRICA Y DE LA ADOLESCENCIA',
 'HEMATO-ONCOLOGIA PEDIATRICA',
 'HEMATOLOGIA',
 'IMAGENOLOGÍA',
 'INFECTOLOGIA',
 'INFECTOLOGIA PEDIATRICA',
 'INMUNOLOGIA',
 'MEDICINA FAMILIAR',
 'MEDICINA FAMILIAR DEL NIÑO',
 'MEDICINA FISICA Y REHABILITACION (FISIATRIA ADULTO)',
 'MEDICINA FISICA Y REHABILITACIO

In [46]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import unicodedata

# -----------------------------
# Utilidad: normalizar strings (quita tildes, mayúsculas, etc.)
# -----------------------------
def normalizar_texto(s: str) -> str:
    if s is None:
        return ""
    s = str(s).strip().upper()
    s = unicodedata.normalize("NFD", s)
    s = "".join(ch for ch in s if unicodedata.category(ch) != "Mn")  # quita tildes
    return s

# -----------------------------
# (A) Función: brechas por centros (robusta a tildes)
# -----------------------------
def graficar_brechas_centros(especialidad, anio, cubo):

    # Normalizar especialidad input y columna
    esp_in = normalizar_texto(especialidad)
    df = cubo.copy()
    df["_esp_norm"] = df["Especialidad"].apply(normalizar_texto)

    # Filtros
    df = df[(df["_esp_norm"] == esp_in) & (df["Año"] == anio)].copy()

    # Validación clave para evitar cols=0
    if df.empty:
        disponibles = sorted(cubo["Especialidad"].dropna().unique().tolist())
        raise ValueError(
            f"No hay datos para especialidad='{especialidad}' (normalizada='{esp_in}') y año={anio}.\n"
            f"Ejemplos disponibles: {disponibles[:10]} ..."
        )

    # Agrupar por Trimestre y Centro
    df = df.groupby(["Trimestre", "Centro_siglas"], as_index=False).agg(
        Oferta_consultas=("Oferta_consultas", "sum"),
        Consultas_solicitadas=("Consultas_solicitadas", "sum"),
        Lista_espera_inicial=("Lista_espera_inicial", "first")  # igual que tu ejemplo
    )

    # Centros
    centros = df["Centro_siglas"].dropna().unique()
    if len(centros) == 0:
        raise ValueError("No hay centros (Centro_siglas) para graficar en este filtro.")

    # Subplots
    fig = make_subplots(
        rows=1,
        cols=len(centros),
        shared_yaxes=True,
        subplot_titles=[f"{c}" for c in centros]
    )

    # Trazas por centro
    for i, centro in enumerate(centros):
        df_c = df[df["Centro_siglas"] == centro].copy().sort_values("Trimestre")

        # Acumulados
        df_c["Demanda_acumulada"] = df_c["Consultas_solicitadas"].cumsum()
        df_c["Oferta_acumulada"] = df_c["Oferta_consultas"].cumsum()

        fig.add_trace(
            go.Scatter(
                x=df_c["Trimestre"], y=df_c["Oferta_acumulada"],
                mode="lines+markers", name="Oferta_acumulada",
                showlegend=(i == 0)
            ),
            row=1, col=i+1
        )

        fig.add_trace(
            go.Scatter(
                x=df_c["Trimestre"], y=df_c["Lista_espera_inicial"],
                fill="tozeroy", name="Lista_espera_inicial",
                stackgroup="one", showlegend=(i == 0)
            ),
            row=1, col=i+1
        )

        fig.add_trace(
            go.Scatter(
                x=df_c["Trimestre"], y=df_c["Demanda_acumulada"],
                fill="tozeroy", name="Demanda_acumulada",
                stackgroup="one", showlegend=(i == 0)
            ),
            row=1, col=i+1
        )

    fig.update_layout(title_text=f"Brechas por Trimestre y Centro ({especialidad} - {anio})")
    fig.show()

# -----------------------------
# (1) Desafío: ejecutar para otra especialidad
#     (Ejemplo: OTORRINOLARINGOLOGIA, que existe en tu lista)
# -----------------------------
graficar_brechas_centros("OTORRINOLARINGOLOGIA", 2021, cubo)
# También debería funcionar aunque le pongas tilde:
# graficar_brechas_centros("OTORRINOLARINGOLOGÍA", 2021, cubo)


# -----------------------------
# (2) Desafío: modificar visualización incorporando otra variable del cubo
#     Ejemplo: graficar brecha acumulada (Oferta - Demanda) y colorear por Servicio
# -----------------------------
def brecha_acumulada_por_servicio(especialidad, anio, cubo):
    esp_in = normalizar_texto(especialidad)
    df = cubo.copy()
    df["_esp_norm"] = df["Especialidad"].apply(normalizar_texto)
    df = df[(df["_esp_norm"] == esp_in) & (df["Año"] == anio)].copy()

    if df.empty:
        raise ValueError(f"No hay datos para {especialidad} y año {anio}")

    # Agregamos por Periodo y Servicio
    df = df.groupby(["Periodo", "Servicio"], as_index=False).agg(
        Oferta=("Oferta_consultas", "sum"),
        Demanda=("Consultas_solicitadas", "sum")
    )

    # Brecha acumulada por servicio
    df = df.sort_values(["Servicio", "Periodo"])
    df["Oferta_acum"] = df.groupby("Servicio")["Oferta"].cumsum()
    df["Demanda_acum"] = df.groupby("Servicio")["Demanda"].cumsum()
    df["Brecha_acum"] = df["Oferta_acum"] - df["Demanda_acum"]

    fig = px.line(
        df,
        x="Periodo",
        y="Brecha_acum",
        color="Servicio",
        title=f"Brecha acumulada (Oferta - Demanda) por Servicio ({especialidad} - {anio})"
    )
    fig.show()

brecha_acumulada_por_servicio("DERMATOLOGIA", 2021, cubo)


# -----------------------------
# (3) Desafío: otra visualización interesante
#     Ejemplo: Brecha trimestral (Oferta - Demanda) tipo barras
# -----------------------------
def brecha_trimestral_barras(especialidad, anio, cubo):
    esp_in = normalizar_texto(especialidad)
    df = cubo.copy()
    df["_esp_norm"] = df["Especialidad"].apply(normalizar_texto)
    df = df[(df["_esp_norm"] == esp_in) & (df["Año"] == anio)].copy()

    if df.empty:
        raise ValueError(f"No hay datos para {especialidad} y año {anio}")

    df = df.groupby(["Trimestre"], as_index=False).agg(
        Oferta=("Oferta_consultas", "sum"),
        Demanda=("Consultas_solicitadas", "sum")
    )
    df["Brecha"] = df["Oferta"] - df["Demanda"]

    fig = px.bar(
        df,
        x="Trimestre",
        y="Brecha",
        title=f"Brecha trimestral (Oferta - Demanda) ({especialidad} - {anio})"
    )
    fig.add_hline(y=0)
    fig.show()

brecha_trimestral_barras("OTORRINOLARINGOLOGIA", 2021, cubo)


## 🏁 Discusión

1. ¿Qué tendría que hacer para reproducir el análisis con datos de 2024?
4. ¿Qué tendría que hacer para replicar el análisis en otro Servicio de Salud?
4. ¿Qué tendría que hacer si necesitara incorporar variables que no están en el cubo (ejemplo: solicitudes de interconsultas GES)?